In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

# from sklearn import preprocessing
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.model_selection import GridSearchCV
# from sklearn import tree


import numpy as np

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


In [2]:
df_init = pd.read_csv('bank-full.csv', sep = ';')

In [3]:
df_init

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [4]:
# Removing column y for further analysis
data = df_init.drop(columns = 'y')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
dtypes: int64(7), object(9)
memory usage: 5.5+ MB


In [5]:
# One-hot-encoding
encoded_data = pd.get_dummies(data=data, drop_first=True)

In [9]:
# Creating an instanceStandardScaler
scaler = StandardScaler()

# Encode the data
df_scaled = scaler.fit_transform(encoded_data)

# Create a new DataFrame with the scaled data
df_scaled = pd.DataFrame(df_scaled, columns=encoded_data.columns)

In [10]:
# Create a PCA object while retaining 95% of variance
pca = PCA(0.95)

# Fitting the PCA model to the data and data transformation
reduced_data = pca.fit_transform(df_scaled)

In [11]:
# input data reconstruction
reconstructed_data = pca.inverse_transform(reduced_data)

In [12]:
type(reconstructed_data)

numpy.ndarray

In [13]:
print(reconstructed_data.shape)
print(encoded_data.shape)

(45211, 42)
(45211, 42)


In [14]:
difference_data = df_scaled - reconstructed_data
difference_data

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,0.977304,-0.062487,-0.478014,0.002792,0.220932,0.037856,-0.033507,0.184057,-0.007420,-0.045865,...,0.240594,-0.396636,0.002850,-0.002389,0.188693,0.060154,-0.022411,0.021792,-0.001924,-0.037528
1,0.693877,-0.045668,-0.476104,0.000300,0.219952,0.045861,-0.038251,0.171689,0.040267,-0.013538,...,0.238781,-0.400123,0.002134,0.003051,0.202325,0.056826,-0.028559,0.005978,-0.013103,-0.068699
2,-0.516955,0.029376,-0.244081,-0.007606,0.105544,0.099771,-0.028872,0.069882,0.071315,0.095335,...,0.122338,-0.274710,-0.003966,-0.059393,0.147259,0.013980,-0.033933,-0.045946,-0.045555,-0.099038
3,0.058711,-0.010600,-0.360932,-0.005744,0.168783,0.059753,-0.030084,0.151040,-0.019806,0.060875,...,0.186092,-0.304297,-0.005354,0.012511,0.183228,0.032326,-0.029713,-0.022692,-0.043383,-0.091876
4,-0.683487,0.031406,-0.543451,-0.019212,0.286724,0.097888,-0.037398,0.038944,0.021678,0.100108,...,0.145742,-0.387186,-0.043302,0.127424,0.241250,0.013746,-0.084843,-0.054148,-0.063661,-0.104637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.542160,-0.047879,-0.679104,-0.016809,0.394740,0.120743,-0.001870,0.079590,-0.080463,-0.038870,...,0.136805,-0.233672,-0.056089,0.423686,0.233715,0.042740,-0.075835,0.043909,0.034778,0.222679
45207,-0.123942,0.018824,-0.367496,-0.009554,0.216056,0.154802,0.008857,-0.072326,0.009250,-0.066936,...,0.060047,-0.119846,-0.018381,0.238460,0.144332,0.039596,-0.033752,0.022687,0.054030,0.216572
45208,-0.211553,-0.000857,-0.475584,-0.016127,0.253694,0.306444,-0.002535,0.119497,0.016425,0.091940,...,0.172099,-0.238500,-0.023613,0.226750,0.257628,0.037766,-0.053503,0.003883,0.017000,0.245677
45209,0.455513,-0.039697,-0.475810,-0.011153,0.257859,0.125934,-0.006100,0.217708,-0.019861,0.038870,...,0.152647,-0.226012,-0.024898,0.216663,0.190310,0.040323,-0.042448,0.026446,0.016981,0.143061


In [15]:
type(difference_data)

pandas.core.frame.DataFrame

In [16]:
l2_distance = np.linalg.norm(difference_data.values, axis = 1)

In [17]:
df_distance = pd.DataFrame(l2_distance)
df_distance.sort_values(by = 0,inplace = True, ascending = False)
PCA_anomalies = df_distance.head(round(len(df_distance)/20))

# DBSCAN 

In [18]:
# Create a DBSCAN instance
dbscan = DBSCAN(eps=4, min_samples=7)

# Fit the DBSCAN model to the data
labels = dbscan.fit_predict(df_scaled)
count = np.count_nonzero(labels == -1)

print("Percentage of values equal to -1:", count/len(labels))

Percentage of values equal to -1: 0.052708411669726396


In [19]:
# select anomalies
df_distance_DBSCAN = pd.DataFrame(labels)
DBSCAN_anomalies = df_distance_DBSCAN[df_distance_DBSCAN[0]==-1]

In [23]:
# compare anomalies detected by PCA and DBSCAN
series_PCA = pd.Series(PCA_anomalies.index)
series_DBSCAN = pd.Series(DBSCAN_anomalies.index)
common_anomalies = series_PCA[series_PCA.isin(series_DBSCAN)]
common_anomalies_number = len(common_anomalies)

print("Number of anomalies detected by both PCA and DBSCAN:")
print(common_anomalies_number)  # Konwertuj do listy

Number of anomalies detected by both PCA and DBSCAN:
344


In [24]:
print(f'% of PCA anomalies detected by DBSCAN: {round(common_anomalies_number / len(series_PCA) * 100, 2)}%')
print(f'% of DBSCAN anomalies detected by PCA: {round(common_anomalies_number / len(series_DBSCAN) * 100, 2)}%')

% of PCA anomalies detected by DBSCAN: 15.21%
% of DBSCAN anomalies detected by PCA: 14.44%
